# Homework 2 - Data from the web

### Useful Imports

In [1]:
# Import libraries
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import sys
import re 

# those two packages must be installed using for example ```pip install editdistance```
import editdistance
import unidecode
sns.set_context('notebook')
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

## 1 - Top Universities

In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total)

In [2]:
ROOT_URL_TOP_UNIV = "https://www.topuniversities.com/"

In [3]:
def progress(count, total, suffix=''):
    """ Shows the progress of a given action 
    
    @params:
    - count : the current count of done operations
    - total : the total number of operation to do
    - suffix : a message printed after the progress bar
    """
    
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '#' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ... %s\r' % (bar, percents, '%', suffix))
    sys.stdout.flush()  # As suggested by Rom Ruben

In [4]:
def get_number(enclosing_class_name,url,det_soup):
    """Tries to extract the number we look for. Will return a NaN 
    if the details page doesn't contain such a number
    
    @params:
    - enclosing_class_name : the class of the div into which the number is contained.
    - url : the suffix url of the school (to print an error if not found)
    - det_soup : the soup of the details page for the given school
    
    """
    try:
        n = det_soup.find("div",class_=enclosing_class_name).find("div",class_="number").text.strip('\n')
    except(AttributeError) as e:
        print("Couldn't find '{}' for {}".format(enclosing_class_name,url))
        n = 'NAN'
    return n

def get_details(url):
    """Finds the detail of a given university on the details page
    
    @params:
    - url : the suffix url of the universirty of interest
    
    """
    details_url = ROOT_URL_TOP_UNIV + url
    det_request = requests.get(details_url)
    det_soup = BeautifulSoup(det_request.text, 'lxml')
    labels = ['total faculty','inter faculty','total student','total inter']

    staff_total,staff_inter,student_total,student_inter = [get_number(label,url,det_soup) for label in labels]
    
    return staff_total,staff_inter,student_total,student_inter

In [5]:
def get_ranking_topUni(path_to_dump):
    r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508057903494')
    if(os.path.isfile(path_to_dump)):
        return pd.read_pickle(path_to_dump)

    top_200 = []

    for (i,uni_dict) in enumerate(r.json()['data']):
        if(i<200):
            title = uni_dict['title']
            rank = uni_dict['rank_display']
            country = uni_dict['country']
            region = uni_dict['region']
            url = uni_dict['url']

            # In case of tie, the rank is prefixed with an '=', we get rid of it.
            rank = rank.strip('=')

            # We then get the details from the details page
            staff_total,staff_inter,student_total,student_inter = get_details(url)
            progress(i+1,200,'Scraping the info')

            top_200.append({'School Name': title, 
                            'Rank': rank, 
                            'Country': country,
                            'Region': region,
                            'Total Staff': staff_total,
                            'International Staff': staff_inter,
                            'Total Student': student_total,
                            'International Student': student_inter
                           })
            
    df = pd.DataFrame.from_dict(top_200)
    
    
    # We set Rank as an index
    df.set_index('Rank',inplace=True)
    
    # Then we convert the number columns to integers
    cols = ['Total Staff','International Staff','Total Student','International Student']
    df[cols] = df[cols].apply(lambda x: pd.to_numeric(x.astype(str)
                                                       .str.replace(',',''), errors='coerce',downcast='integer'))

    # We return the dataframe ordered as we please
    df = df[['School Name','Total Staff','International Staff','Total Student','International Student','Country','Region']]
    
    # We serialize it using pickle so that we do not have to download it again
    df.to_pickle(path_to_dump)
    return df

In [6]:
df_topUni = get_ranking_topUni('topuniversities_backup')
df_topUni.head()

,School Name,Total Staff,International Staff,Total Student,International Student,Country,Region
Rank,,,,,,,
1,Massachusetts Institute of Technology (MIT),2982,1679.0,11067,3717,United States,North America
2,Stanford University,4285,2042.0,15878,3611,United States,North America
3,Harvard University,4350,1311.0,22429,5266,United States,North America
4,California Institute of Technology (Caltech),953,350.0,2255,647,United States,North America
5,University of Cambridge,5490,2278.0,18770,6699,United Kingdom,Europe


### 1.a - Best university in terms of ratio faculty members over students

In order to determine which is the best university regarding ratio between faculty members and students, we create a new column called ```Fac/stud ratio```. Then we sort the rows according to values in this column. (because we were asked for the best universities we display the ranking according to this value)

In [7]:
df_topUni['Fac/stud ratio'] = df_topUni['Total Staff']/df_topUni['Total Student']
sorted_by_facStudRatio = df_topUni.sort_values('Fac/stud ratio',ascending=False)
sorted_by_facStudRatio.head()

,School Name,Total Staff,International Staff,Total Student,International Student,Country,Region,Fac/stud ratio
Rank,,,,,,,,
4,California Institute of Technology (Caltech),953,350.0,2255,647,United States,North America,0.422616
16,Yale University,4940,1708.0,12402,2469,United States,North America,0.398323
6,University of Oxford,6750,2964.0,19720,7353,United Kingdom,Europe,0.342292
5,University of Cambridge,5490,2278.0,18770,6699,United Kingdom,Europe,0.292488
17,Johns Hopkins University,4462,1061.0,16146,4105,United States,North America,0.276353


### 1.b - Best university in terms of ratio of international students

We use strategy that is exactly the same than for the 1.a

In [8]:
df_topUni['International ratio'] = df_topUni['International Student']/df_topUni['Total Student']
sorted_by_internationalRatio = df_topUni.sort_values('International ratio',ascending=False)
sorted_by_internationalRatio.head()

,School Name,Total Staff,International Staff,Total Student,International Student,Country,Region,Fac/stud ratio,International ratio
Rank,,,,,,,,,
35,London School of Economics and Political Scien...,1088,687.0,9760,6748,United Kingdom,Europe,0.111475,0.691393
12,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695,1300.0,10343,5896,Switzerland,Europe,0.163879,0.570047
8,Imperial College London,3930,2071.0,16090,8746,United Kingdom,Europe,0.244251,0.543567
200,Maastricht University,1277,502.0,16385,8234,Netherlands,Europe,0.077937,0.502533
47,Carnegie Mellon University,1342,425.0,13356,6385,United States,North America,0.100479,0.478062


### 1.c - Aggregated by country

Because it wasn't so clear how we should aggregate the data, we decided to consider the ```mean()``` as aggregation function which will give us the ranking of countries according to :
* ratio of faculty staff over students
* ratio of international students

In [9]:
best_countries_by_facStudRatio = df_topUni.groupby('Country').mean().sort_values('Fac/stud ratio',ascending=False)
best_countries_by_facStudRatio[['Fac/stud ratio']].head()

,Fac/stud ratio
Country,
Russia,0.221910
Denmark,0.186580
Saudi Arabia,0.175828
Singapore,0.162279
Japan,0.155840


In [10]:
best_countries_by_internationalRatio = df_topUni.groupby('Country').mean().sort_values('International ratio',ascending=False)
best_countries_by_internationalRatio[['International ratio']].head()

,International ratio
Country,
United Kingdom,0.351308
Australia,0.346878
Switzerland,0.313816
Hong Kong,0.312148
Austria,0.306095


### 1.d - Aggregated by region

Very similar strategy than the one used in 1.c

In [11]:
best_regions_by_facStudRatio = df_topUni.groupby('Region').mean().sort_values('Fac/stud ratio',ascending=False)
best_regions_by_facStudRatio[['Fac/stud ratio']].head()

,Fac/stud ratio
Region,
North America,0.145407
Asia,0.134673
Europe,0.120003
Latin America,0.096779
Africa,0.088450


In [12]:
best_regions_by_internationalRatio = df_topUni.groupby('Region').mean().sort_values('International ratio',ascending=False)
best_regions_by_internationalRatio[['International ratio']].head()

,International ratio
Region,
Oceania,0.329077
Europe,0.245932
North America,0.203583
Africa,0.169703
Asia,0.132394


# todo : bar plots for the results

## 2 - Times Higher Education

Because we do not have direct access to some informations (e.g. International Students, Total Staff) we will infer them using the informations we have.

In [13]:
def get_ranking_times(path_to_dump):
    if(os.path.isfile(path_to_dump)):
        return pd.read_pickle(path_to_dump)
    
    r = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')

    top_200 = []
    for (i,uni_dict) in enumerate(r.json()['data']):
        if(i<200):
            title = uni_dict['name']
            # We delete the '=' sign in the case of tie to be consistent with the first one
            rank = uni_dict['rank'].replace('=','')
            country = uni_dict['location']

            # We delete the thousands separator
            student_total = int(uni_dict['stats_number_students'].replace(',',''))
            pc_inter_student = float(uni_dict["stats_pc_intl_students"].replace('%',''))/100
            student_to_staff_ratio = float(uni_dict["stats_student_staff_ratio"])

            top_200.append({'School Name': title, 
                                'Rank': rank, 
                                'Country': country,
                                'Student to Staff ratio': student_to_staff_ratio,
                                'Total Student': student_total,
                                'Perc. International Student': pc_inter_student
                               })
    df = pd.DataFrame.from_dict(top_200)
    df.to_pickle(path_to_dump)
    return df

In [14]:
df_times = get_ranking_times('times_backup')
df_times.head()

,Country,Perc. International Student,Rank,School Name,Student to Staff ratio,Total Student
0,United Kingdom,0.38,1,University of Oxford,11.2,20409
1,United Kingdom,0.35,2,University of Cambridge,10.9,18389
2,United States,0.27,3,California Institute of Technology,6.5,2209
3,United States,0.22,3,Stanford University,7.5,15845
4,United States,0.34,5,Massachusetts Institute of Technology,8.7,11177


## 3 - Merging the two data frames
### 3.1 - Matching the school names

Because we do not have the same information than in the first table we try to get the informations using the first table. But first we need to see how the school names can differ between the two tables (e.g. "Massachusetts Institute of Technology (MIT)" is different from "Massachusetts Institute of Technology"). 
* First we get rid of the accents of the school name because these can differ from one ranking to another.
* Secondly we delete the terms into parenthesis, they usually represent the Initials of the school.
* Then we try to identify names that are highly similar

In [15]:
def how_different(df_times,df_topUni):
    """ 
    Will return the intersection set of school names from the two dataframes. 
    This helps us visualise what names are considered different between the two
    """
    names_times = set(sorted(list(df_times['School Name'].values)))
    names_topUni = set(sorted(list(df_topUni['School Name'].values)))
    difference = names_times.symmetric_difference(names_topUni)
    return difference

In [16]:
def clean_data(x):
    """
    Performs very basic string cleaning on a serie of School names.
    It puts all names in lower case, gets rid of the accents,
    deletes the symbols that are added between parenthesis (like EPFL),
    the trailing "the" and some ponctuation.
    """
    # get small latters
    x = x.lower()
    
    # delete accents
    x = unidecode.unidecode(x)
    
    # delete parenthesis, points, comas, dash and trailing "the"
    x = re.sub("[.,]","",x)
    x = re.sub(r"(\s)\-(\s)", r"\1\2",x) #dash between two spaces replace by nothing
    x = re.sub(r"([a-z])\-([a-z])", r"\1 \2",x) #dash between two words replace by space
    x = re.sub("[\(].*?[\)]", "", x)
    x = re.sub("^(the)\s", "",x)
    
    # delete any space before or after string
    return x.strip()

print("Before data cleaning, we have {} different school names.".format(len(how_different(df_times,df_topUni))))
df_times['School Name'] = df_times['School Name'].apply(clean_data)
df_topUni['School Name'] = df_topUni['School Name'].apply(clean_data)
print("After data cleaning, we have {} different school names.".format(len(how_different(df_times,df_topUni))))

Before data cleaning, we have 190 different school names.
After data cleaning, we have 116 different school names.


In [17]:
def get_difference(s1,s2):
    # Many name of school contain same words "technology","university", 
    # we want to get rid of them such that the relative difference 
    # really characterises words that are different
    d1 = [w for w in s1.split() if w not in s2]
    d2 = [w for w in s2.split() if w not in s1]
    return " ".join(d1)," ".join(d2)

def check_similarity(s1,s2,threshold,combined):
    o,n = get_difference(s1,s2)
    same_country = (combined.loc[s1,'Country'] == combined.loc[s1,'Country'])
    
    if(len(o) == 0 or len(n)==0):
        # One school name was contained in the other
        return same_country
    else:
        max_length =  len(n) if (len(n) > len(o)) else len(o)
        relative_diff = float(editdistance.eval(o,n))/max_length

        # Then we compute some of our conditions that determine if the strings are similar or not.
        similar_ortho = (relative_diff < threshold)

        return (similar_ortho and same_country)

def get_potentially_similar(df_times,df_topUni):
    reduced_times = df_times.set_index('School Name')[['Country']].reset_index()
    reduced_topUni = df_topUni.set_index('School Name')[['Country']].reset_index()
    
    # We want a dataframe giving us the country for each school
    combined = reduced_times.merge(reduced_topUni,how='outer').set_index('School Name')
    difference = how_different(df_times,df_topUni)
    
    potentially_similar = []
    for i in range(len(difference)):
        my_list = []
        n = list(difference)[i]
        my_list.append(n)
        for j in range(i+1,len(difference)):
            o = list(difference)[j]
            if(o!=n):
                # we check different conditions
                if(check_similarity(o,n,0.35,combined)):
                    my_list.append(o)
        if(len(my_list) > 1):
            potentially_similar.append(my_list)
    return potentially_similar

def make_homogeneous(x,to_homogeneous):
        if(x in to_homogeneous.keys()): return to_homogeneous[x]
        else: return x

def match_school_names(df_times,df_topUni):
    potentially_similar = get_potentially_similar(df_times,df_topUni)
    
    # we arbitrarily chose one of the names as the universal one, 
    # which gives us a dict that we can apply to our dataframes.
    to_homogeneous = {}
    for sublist in potentially_similar:
        for elem in sublist:
            to_homogeneous[elem] = sublist[0]

    df_times['School Name'] = df_times['School Name'].apply(make_homogeneous,args=(to_homogeneous,))
    df_topUni['School Name'] = df_topUni['School Name'].apply(make_homogeneous,args=(to_homogeneous,))

In [18]:
print("Before matching names, we have {} different school names.".format(len(how_different(df_times,df_topUni))))
match_school_names(df_times,df_topUni)
print("After matching names, we have {} different school names.".format(len(how_different(df_times,df_topUni))))

Before matching names, we have 116 different school names.
After matching names, we have 94 different school names.


### 3.2 - Merging

Now that we have only so few different names we can consider that the rest are really different from the two rankings. Indeed we have performed all the matching that we could. We can now merge the two dataframes. We have a few remarks :
* We only keep the rankings for the df_time because df_topUni contains more informations and the information that are common to both should be identical (or at least very close)

In [20]:
df_times_to_merge = df_times[['School Name','Rank']]
df_topUni_to_merge = df_topUni.reset_index()
merged_df = df_topUni_to_merge.merge(df_times_to_merge,on='School Name',how='outer', suffixes=('_topUni', '_times'))
merged_df.head()

,Rank_topUni,School Name,Total Staff,International Staff,Total Student,International Student,Country,Region,Fac/stud ratio,International ratio,Rank_times
0,1,massachusetts institute of technology,2982.0,1679.0,11067.0,3717.0,United States,North America,0.269450,0.335863,5
1,2,stanford university,4285.0,2042.0,15878.0,3611.0,United States,North America,0.269870,0.227422,3
2,3,harvard university,4350.0,1311.0,22429.0,5266.0,United States,North America,0.193945,0.234785,6
3,4,california institute of technology,953.0,350.0,2255.0,647.0,United States,North America,0.422616,0.286918,3
4,5,university of cambridge,5490.0,2278.0,18770.0,6699.0,United Kingdom,Europe,0.292488,0.356899,2


TODO: infer informations from the stats of times to get the same kind of information than for topUni to be able to work on a coherent dataset